# rst2ipynb

> Minimal .rst to .ipynb converter

In [ ]:
#|default_exp rst2ipynb

In [ ]:
#|export
import json
from docutils import nodes
from docutils.parsers.rst.roles import register_local_role
from docutils.parsers.rst import Parser
from docutils.frontend import OptionParser
from docutils.utils import new_document

from fastcore.script import *
from fastcore.utils import *
from execnb.nbio import *

In [ ]:
#|export
def ref_role(name, rawtext, text, lineno, inliner, options=None, content=None):
    "Custom `ref` role."
    node = nodes.reference(rawtext, text.split(' ')[0])
    return [node], []

In [ ]:
#|export
register_local_role('ref', ref_role)

In [ ]:
#|export
def parse_rst(fname):
    "Parse a .rst file."
    text = Path(fname).read_text()
    parser = Parser()
    components = (Parser,)
    settings = OptionParser(components=components).get_default_values()
    document = new_document('<rst-doc>', settings=settings)
    parser.parse(text, document)
    return document

In [ ]:
#|hide
# doc = parse_rst('/Users/seem/code/nbdev-sqlite-utils-bak/python-api.rst')

In [ ]:
#|export
class NotebookVisitor(nodes.NodeVisitor):
    "Convert an rst document into notebook cells."
    def __init__(self, *args, **kwargs):
        self.cells = []
        self._lvl = 0
        super().__init__(*args, **kwargs)
        
    def _cell(self, d):
        d['source'] = d['source'].splitlines(True)
        self.cells.append(d)
    def _markdown(self, s): self._cell({'cell_type':'markdown','metadata':{},'source':s})
    def _code(self, s): self._cell({'cell_type':'code','execution_count':None,'metadata':{},'outputs':[],'source':s})

    def visit_section(self, node): self._lvl += 1
    def depart_section(self, node): self._lvl -= 1

    def visit_title(self, node):
        s = '#'*self._lvl + ' ' + node.astext()
        self._markdown(s)

    def visit_paragraph(self, node):
        s = ''
        for n in node:
            if isinstance(n, (nodes.literal,nodes.reference,nodes.title_reference)): s += f'`{n.astext()}`'
            else: s += str(n)
        self._markdown(s)

    def visit_literal_block(self, node):
        self._code(node.astext())

    def unknown_visit(self, node): pass
    def unknown_departure(self, node): pass

In [ ]:
#|export
def _nb(cells): return {'cells': cells, 'nbformat': 4, 'nbformat_minor': 4, 'metadata': {}}

In [ ]:
#|export
@call_parse
def rst2ipynb(fname:str): # Path to .rst file
    "Convert a .rst file to .ipynb."
    doc = parse_rst(fname)
    vis = NotebookVisitor(doc)
    doc.walkabout(vis)
    nb = _nb(vis.cells)
    out = Path(fname).with_suffix('.ipynb')
    write_nb(nb, out)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()